In [1]:
import gurobipy as gp
from gurobipy import GRB
import sys

# Data Processing
import pandas as pd
import numpy as np

In [2]:
tabla_final = pd.read_csv('Resultados_Regresion_Logistica.csv', sep = ",")
tabla_final = tabla_final[["ID_CLIENTE","probabilidad_de_aceptar"]]
print(tabla_final)

        ID_CLIENTE  probabilidad_de_aceptar
0          98717.0                 0.781240
1          48404.0                 0.736765
2          19398.0                 0.734941
3          16233.0                 0.731999
4          19929.0                 0.729799
...            ...                      ...
602555   1436134.0                 0.010711
602556   1402635.0                 0.010709
602557   1440072.0                 0.010707
602558   1400657.0                 0.010706
602559   1401811.0                 0.010705

[602560 rows x 2 columns]


In [3]:
array_cliente_probabilidad = tabla_final.to_numpy()
array_cliente_probabilidad = array_cliente_probabilidad.tolist()

In [4]:
# Crear un objeto de modelo de Gurobi
model = gp.Model()

Restricted license - for non-production use only - expires 2024-10-28


In [6]:
# Conjuntos del modelo

I = array_cliente_probabilidad #Conjunto de personas preaprobadas

In [7]:
# Obtenemos las cardinalidades de cada conjunto
car_i = len(I)

In [8]:
# Parámetros del modelo
C = 1000 # Cobertura

p = list() # puntaje de posibilidad de la adquisición de una tarjeta de crédito para la persona preaprobada i.
for i in range(car_i):
    p.append(array_cliente_probabilidad[i][1])
U = 29587 #utilidades generadas por una persona
CPA_base = 8238

In [9]:
# Creamos las variables de decisión

e = model.addVars(car_i, vtype=GRB.BINARY, name="e")  # 1 si a la persona se le envia cualquier tipo de publicidad, 0 en otro caso.

In [10]:
# Seteamos la función objetivo

model.setObjective((gp.quicksum(e[i] * p[i] for i in range(car_i)))/(gp.quicksum(e[i] for i in range(car_i))), sense=GRB.MAXIMIZE)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "src/gurobipy/linexpr.pxi", line 568, in gurobipy.LinExpr.__truediv__
TypeError: float() argument must be a string or a number, not 'gurobipy.LinExpr'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Cristobal/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/4f/dbgwrrz13h1_vh4b2d4htbn80000gn/T/ipykernel_20784/674696661.py", line 3, in <module>
    model.setObjective((gp.quicksum(e[i] * p[i] for i in range(car_i)))/(gp.quicksum(e[i] for i in range(car_i))), sense=GRB.MAXIMIZE)
  File "src/gurobipy/linexpr.pxi", line 570, in gurobipy.LinExpr.__truediv__
gurobipy.GurobiError: Divisor must be a constant

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Cristobal/Library/Python/3.9/lib

In [11]:
# Agregamos las restricciones

# Cobertura fija
model.addConstr(gp.quicksum(e[i] for i in range(car_i)) == C, "c1")
# CPA < Ingreso
model.addConstrs(123.36 * ( (gp.quicksum(e[i] * p[i] for i in range(car_i)))/(gp.quicksum(e[i] for i in range(car_i))))^(-0.994) <= U, "c2")
# Limitar el número de ofertas enviadas a una persona
model.addConstrs(123.36 * ( (gp.quicksum(e[i] * p[i] for i in range(car_i)))/(gp.quicksum(e[i] for i in range(car_i))))^(-0.994) >= CPA_base , "c3")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "src/gurobipy/linexpr.pxi", line 568, in gurobipy.LinExpr.__truediv__
TypeError: float() argument must be a string or a number, not 'gurobipy.LinExpr'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Cristobal/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/4f/dbgwrrz13h1_vh4b2d4htbn80000gn/T/ipykernel_20784/3660822398.py", line 6, in <module>
    model.addConstrs(123.36 * ( (gp.quicksum(e[i] * p[i] for i in range(car_i)))/(gp.quicksum(e[i] for i in range(car_i))))^(-0.994) <= U, "c2")
  File "src/gurobipy/linexpr.pxi", line 570, in gurobipy.LinExpr.__truediv__
gurobipy.GurobiError: Divisor must be a constant

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Cristobal/Library/Pyt

In [12]:
# Optimizamos el modelo

model.optimize()

# Imprimimos la solución óptima

if model.status == GRB.OPTIMAL:
    print("Encontrada solución óptima!")
    print("Valor de función objetivo: ", model.objVal)
    print("Variables:")
    for var in model.getVars():
        print(var.varName, var.x)
else:
    print("No se encontró solución factible.")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [82]:
valores_e = [e[i].x for i in range(car_i)]
df = pd.DataFrame({'Personas_que_se_envia':valores_e})
df

,Personas_que_se_envia
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
602555,0.0
602556,0.0
602557,0.0
602558,0.0


In [83]:
tabla_con_id_cliente_mas_probabilidad = pd.merge(tabla_final, df, left_index=True, right_index=True)

tabla_con_id_cliente_mas_probabilidad

,ID_CLIENTE,probabilidad_de_aceptar,Personas_que_se_envia
0,98717.0,0.781240,1.0
1,48404.0,0.736765,1.0
2,19398.0,0.734941,1.0
3,16233.0,0.731999,1.0
4,19929.0,0.729799,1.0
...,...,...,...
602555,1436134.0,0.010711,0.0
602556,1402635.0,0.010709,0.0
602557,1440072.0,0.010707,0.0
602558,1400657.0,0.010706,0.0


In [84]:
valores_positivos = tabla_con_id_cliente_mas_probabilidad[tabla_con_id_cliente_mas_probabilidad['Personas_que_se_envia'] == 1]
valores_positivos

,ID_CLIENTE,probabilidad_de_aceptar,Personas_que_se_envia
0,98717.0,0.781240,1.0
1,48404.0,0.736765,1.0
2,19398.0,0.734941,1.0
3,16233.0,0.731999,1.0
4,19929.0,0.729799,1.0
...,...,...,...
23221,149869.0,0.475009,1.0
23222,706635.0,0.475008,1.0
23223,624667.0,0.475004,1.0
23224,417288.0,0.475003,1.0


In [85]:
media = valores_positivos["probabilidad_de_aceptar"].mean()
desviacion = valores_positivos["probabilidad_de_aceptar"].std()
raiz_tamaño_muestra = np.sqrt(len(valores_positivos["probabilidad_de_aceptar"]))

Intervalo_superior = media + (1.96 * desviacion / raiz_tamaño_muestra)
Intervalo_superior = Intervalo_superior * len(valores_positivos["probabilidad_de_aceptar"]) * 31850 - len(valores_positivos["probabilidad_de_aceptar"]) *128.8
Intervalo_superior



406234955.7389619

In [86]:
Intervalo_inferior = media - (1.96 * desviacion / raiz_tamaño_muestra)
Intervalo_inferior = Intervalo_inferior* len(valores_positivos["probabilidad_de_aceptar"]) * 31850 - len(valores_positivos["probabilidad_de_aceptar"]) *128.8
Intervalo_inferior

405235273.76324093

In [87]:
media

0.552521356325351

In [88]:
desviacion

0.052538678799748405